In [19]:
# Optimization of an International Portfolio

In [20]:
! pip install 'pandas_datareader>=0.10'

ERROR: Invalid requirement: "'pandas_datareader"


In [21]:
# Import the necessary libraries 
from pandas_datareader.data import DataReader
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import Dict, List
import yfinance as yf

In [22]:
# Create the formula to get the dates of the last 5 years.
def fnct_last_five_years():
    five_years = timedelta(days=365*5, )
    now = datetime.now()
    five_years_ago = now - five_years
    return now, five_years_ago

now, five_years_ago = fnct_last_five_years()
print(now)
print(five_years_ago)

2022-10-13 12:39:45.150984
2017-10-14 12:39:45.150984


In [23]:
def fnct_get_ticker_data(ticker: str, data_source: str = "yahoo") -> pd.DataFrame:
    """Get the data in the last 5 years for the specified ticker"""
    print(f'downloading data for {ticker}')
    now, five_years_ago = fnct_last_five_years()
    ticker_df = DataReader(ticker, data_source, five_years_ago, now)
    return ticker_df


def fnct_convert_currency(ticker_adj_close:pd.Series, currency:pd.Series) -> pd.Series:
    df = pd.DataFrame({"x":ticker_adj_close, "c":currency})
    # drop rows where all columns in subset are NA
    df = df.dropna(axis=0, how='all', subset=('x', 'c'))
    # fill with the value of the previous date's values
    df = df.fillna(method='ffill')
    # in case there are empty rows at the beginning, fill with following date's values
    df = df.fillna(method='bfill')
    return df['x'] * df['c']


def fnct_get_currency(*tickers):
    """obtain the currency for each ticker"""
    currencies = dict()
    for ticker in tickers:
        t = yf.Ticker(ticker, )
        currencies[ticker] = t.info["currency"]
    return currencies

def fnct_get_currencies_data(currencies:dict):
    """Use the output of get_currency to read the data for all the currencies once"""
    currencies_data = {}
    unique_currencies = set(currencies.values())
    ticker_map = {
        "USD": "MXN=X",
        "CAD": "CADMXN=X",
        "EUR": "EURMXN=X",
    }
    for c in unique_currencies:
        if c == "MXN":
            continue
        exchange = ticker_map[c]
        currencies_data[c] = fnct_get_ticker_data(exchange)['Close'] # {"USD":pd.Series, CAD:pd.Series}
    df = pd.DataFrame(currencies_data)
    df["MXN"] = 1
    return df

def fnct_get_tickers_in_mxn(currencies:List[str]):
    c_df = fnct_get_currencies_data(currencies)
    t_data = {}
    for ticker, coin in currencies.items():
        t = fnct_get_ticker_data(ticker)['Adj Close']
        t_in_mxn = fnct_convert_currency(t, c_df[coin])
        t_data[ticker] = t_in_mxn
    return pd.DataFrame(t_data)

In [24]:
def suma(a, b):
    c = a + b
    return(c)

suma(5, 7)


12

In [25]:
#tickers = ["BIMBOA.MX", "CEMEXCPO.MX", "PE&OLES.MX", "AMXL.MX", "HERDEZ.MX",
#    "BBD-B.TO", "RY.TO", "ENB.TO", 
#    "VWAGY", "SAP", "NSRGY", "EADSY",]
tickers = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 
        'MMC', 'JPM', 'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 
        'PCAR', 'TXT', 'TMO', 'DE', 'MSFT', 'HPQ', 'SEE', 
        'VZ', 'CNP', 'NI', 'T', 'BA']
currencies = fnct_get_currency(*tickers)


In [26]:
currencies

{'JCI': 'USD',
 'TGT': 'USD',
 'CMCSA': 'USD',
 'CPB': 'USD',
 'MO': 'USD',
 'APA': 'USD',
 'MMC': 'USD',
 'JPM': 'USD',
 'ZION': 'USD',
 'PSA': 'USD',
 'BAX': 'USD',
 'BMY': 'USD',
 'LUV': 'USD',
 'PCAR': 'USD',
 'TXT': 'USD',
 'TMO': 'USD',
 'DE': 'USD',
 'MSFT': 'USD',
 'HPQ': 'USD',
 'SEE': 'USD',
 'VZ': 'USD',
 'CNP': 'USD',
 'NI': 'USD',
 'T': 'USD',
 'BA': 'USD'}

In [27]:
data = fnct_get_tickers_in_mxn(currencies)


downloading data for MXN=X
downloading data for JCI
downloading data for TGT
downloading data for CMCSA
downloading data for CPB
downloading data for MO
downloading data for APA
downloading data for MMC
downloading data for JPM
downloading data for ZION
downloading data for PSA
downloading data for BAX
downloading data for BMY
downloading data for LUV
downloading data for PCAR
downloading data for TXT
downloading data for TMO
downloading data for DE
downloading data for MSFT
downloading data for HPQ
downloading data for SEE
downloading data for VZ
downloading data for CNP
downloading data for NI
downloading data for T
downloading data for BA


In [28]:
data

,JCI,TGT,CMCSA,CPB,MO,APA,MMC,JPM,ZION,PSA,...,TMO,DE,MSFT,HPQ,SEE,VZ,CNP,NI,T,BA
Date,,,,,,,,,,,,,,,,,,,,,
2017-10-15,714.255245,1023.543871,645.642178,773.866744,934.283241,742.932267,1484.622336,1652.731954,805.957047,3572.428128,...,3603.396457,2270.376032,1403.765058,365.382892,796.077005,759.646715,479.777123,435.016377,351.170817,4688.684484
2017-10-16,704.915833,1010.160284,637.199932,763.747869,922.066802,733.217882,1465.209809,1631.121270,795.418566,3525.716007,...,3556.279403,2240.689199,1385.409800,360.605243,785.667714,749.713776,473.503685,429.328219,346.579000,4627.376491
2017-10-17,698.864123,1015.194922,637.714839,758.910607,919.268820,737.144458,1465.356898,1635.020587,785.454137,3514.757873,...,3525.933520,2268.204522,1390.776095,358.291693,782.603612,758.055220,476.835766,432.633080,348.768084,4628.669351
2017-10-18,686.692908,1008.410594,630.229801,747.408831,914.408633,732.038056,1459.308117,1634.051895,785.085372,3479.797667,...,3518.906872,2258.827775,1385.060836,359.541380,775.842326,758.643824,475.557670,430.418479,342.261486,4633.764348
2017-10-19,703.232802,1024.960037,648.637475,750.182071,916.995152,730.461788,1486.489311,1651.895823,790.627495,3522.333408,...,3613.444004,2277.157570,1403.879097,367.034360,784.776902,774.807100,482.922257,440.012549,345.382198,4660.642696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-09,1010.303105,3051.339722,586.272267,933.989615,857.676048,851.667123,3048.335374,2122.758315,1008.901026,5681.853753,...,10487.803502,7133.613296,4691.780559,501.546221,881.711823,738.098115,536.998938,490.930413,299.245210,2599.667661
2022-10-10,1010.374964,3052.860050,580.860922,951.551140,866.605419,820.942114,3070.606905,2093.532053,996.018000,5734.830398,...,10186.302773,7172.126954,4571.313772,498.308092,893.923685,730.213764,532.406025,488.936165,297.110462,2630.125447
2022-10-11,1024.346791,3100.128925,575.480249,975.587476,889.907528,812.253835,3057.188640,2045.884417,986.824217,5817.206395,...,10049.032748,7290.419423,4522.977815,497.224575,906.160659,721.156177,533.743887,499.030479,297.170943,2640.025796


In [29]:
pct_change_df = data.pct_change()

In [30]:
ann_mean_return = ((1 + pct_change_df.mean())**252)-1
print(ann_mean_return)

JCI      0.109256
TGT      0.280518
CMCSA    0.027023
CPB      0.087605
MO       0.029266
APA      0.277156
MMC      0.166537
JPM      0.095766
ZION     0.104422
PSA      0.119128
BAX      0.027860
BMY      0.081744
LUV     -0.023232
PCAR     0.112026
TXT      0.105101
TMO      0.243149
DE       0.290230
MSFT     0.285804
HPQ      0.131566
SEE      0.079428
VZ       0.014107
CNP      0.059862
NI       0.060275
T        0.002236
BA       0.006271
dtype: float64


In [31]:
ann_stdev = pct_change_df.std() * (252**(1/2))
print(ann_stdev)

JCI      0.274101
TGT      0.330868
CMCSA    0.277727
CPB      0.276323
MO       0.255147
APA      0.652054
MMC      0.228836
JPM      0.297134
ZION     0.350312
PSA      0.240017
BAX      0.252432
BMY      0.240661
LUV      0.365952
PCAR     0.264337
TXT      0.362133
TMO      0.270039
DE       0.321336
MSFT     0.290048
HPQ      0.351853
SEE      0.322243
VZ       0.207960
CNP      0.298639
NI       0.262208
T        0.242044
BA       0.458872
dtype: float64


In [32]:
pct_change_df.cov()

,JCI,TGT,CMCSA,CPB,MO,APA,MMC,JPM,ZION,PSA,...,TMO,DE,MSFT,HPQ,SEE,VZ,CNP,NI,T,BA
JCI,0.000298,0.000140,0.000149,0.000076,0.000116,0.000236,0.000148,0.000205,0.000206,0.000107,...,0.000127,0.000199,0.000153,0.000196,0.000177,0.000090,0.000152,0.000115,0.000128,0.000244
TGT,0.000140,0.000434,0.000135,0.000107,0.000115,0.000126,0.000133,0.000145,0.000138,0.000100,...,0.000140,0.000157,0.000170,0.000166,0.000126,0.000105,0.000112,0.000112,0.000112,0.000155
CMCSA,0.000149,0.000135,0.000306,0.000080,0.000113,0.000171,0.000130,0.000184,0.000172,0.000103,...,0.000111,0.000162,0.000171,0.000169,0.000139,0.000120,0.000147,0.000135,0.000152,0.000199
CPB,0.000076,0.000107,0.000080,0.000303,0.000106,-0.000016,0.000080,0.000064,0.000040,0.000102,...,0.000081,0.000051,0.000086,0.000049,0.000068,0.000106,0.000094,0.000106,0.000095,0.000016
MO,0.000116,0.000115,0.000113,0.000106,0.000258,0.000150,0.000099,0.000136,0.000133,0.000098,...,0.000080,0.000128,0.000095,0.000114,0.000116,0.000102,0.000118,0.000114,0.000120,0.000158
APA,0.000236,0.000126,0.000171,-0.000016,0.000150,0.001687,0.000159,0.000351,0.000401,0.000039,...,0.000104,0.000318,0.000179,0.000343,0.000227,0.000050,0.000236,0.000102,0.000165,0.000526
MMC,0.000148,0.000133,0.000130,0.000080,0.000099,0.000159,0.000208,0.000165,0.000135,0.000115,...,0.000145,0.000146,0.000166,0.000149,0.000136,0.000093,0.000152,0.000122,0.000108,0.000182
JPM,0.000205,0.000145,0.000184,0.000064,0.000136,0.000351,0.000165,0.000350,0.000326,0.000097,...,0.000119,0.000234,0.000176,0.000225,0.000199,0.000110,0.000189,0.000137,0.000162,0.000324
ZION,0.000206,0.000138,0.000172,0.000040,0.000133,0.000401,0.000135,0.000326,0.000487,0.000070,...,0.000081,0.000240,0.000129,0.000230,0.000192,0.000099,0.000172,0.000123,0.000161,0.000348
PSA,0.000107,0.000100,0.000103,0.000102,0.000098,0.000039,0.000115,0.000097,0.000070,0.000229,...,0.000104,0.000103,0.000118,0.000091,0.000106,0.000095,0.000141,0.000147,0.000093,0.000121


In [33]:

weight = np.full_like(ann_stdev,1/len(ann_stdev))
weight

array([0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04])

In [34]:
portaf_ann_stdev = ((weight @ pct_change_df.cov()) @ weight.T) ** (1/2)
portaf_ann_stdev

0.012627474449949919

In [35]:
pct_change

NameError: name 'pct_change' is not defined

In [ ]:
ticker = 'CADMXN=X'
data_source = 'yahoo'
goog = DataReader(ticker, data_source, five_years_ago, now)
goog

In [ ]:
ticker = 'CEMEXCPO.MX'
data_source = 'yahoo'
cemex = DataReader(ticker, data_source, five_years_ago, now)

In [ ]:
ticker = 'MXN=X'
data_source = 'yahoo'
mxn = DataReader(ticker, data_source, five_years_ago, now)

In [ ]:
df = pd.DataFrame({"goog":goog.Close, "cemex":cemex.Close, 'mxn':mxn['Close']})
# drop rows where all columns in subset are NA
df = df.dropna(axis=0, how='all', subset=('goog', 'cemex'))
# fill with the value of the previous date's values
df = df.fillna(method='ffill')
# in case there are empty rows at the beginning, fill with following date's values
df = df.fillna(method='bfill')

In [ ]:
#Add a column with google prices in MXN
df['goog_mxn'] = df.goog * df.mxn

In [ ]:
df.plot(y=['goog_mxn', "cemex"])

In [ ]:
df.corr()

In [ ]:
df.cov()

In [ ]:
df.std()

In [ ]:
df.std() * np.sqrt(252)

In [ ]:
df['goog_mxn_change'] = df.goog_mxn.pct_change()

In [ ]:
df.to_excel('example.xlsx')